In [1]:
import pandas as pd
import json

In [2]:
def processa_dataset(ano):
    df = pd.read_csv(f"dataset/MICRODADOS_ENEM_{ano}.csv", sep=";", encoding='ansi')

    # provas = ['CN', 'CH', 'LC', 'MT']
    # for i in range(4):
        # print(df[f"TX_RESPOSTAS_{provas[i]}"])
        # print(df[f"TX_GABARITO_{provas[i]}"])

    # print(df.head(10))

In [3]:
# MAPEAMENTO_PROVAS = {
#     '2023': {'LC': 1201, 'CH': 1191, 'CN': 1221, 'MT': 1211}
# }

MAPEAMENTO_PROVAS = {
    '2023': [1201, 1191, 1221, 1211]
}

In [4]:
def processa_prova(ano):
    df = pd.read_csv(f"dataset/ITENS_PROVA_{ano}.csv", sep=";", encoding='ansi')

    #df = df[df['TX_GABARITO'] != 'X'] # Remove questões que foram anuladas
    #df = df[df['SG_AREA'] == 'LC']    # Filtra apenas dados da prova de Linguagens

    #Pega somente as questões do carderno mapeado
    df = df[df['CO_PROVA'].isin(MAPEAMENTO_PROVAS[str(ano)])]

    # Normalizar a dificuldade para o intervalo 1-10
    dificuldade_minima = df['NU_PARAM_B'].min()
    dificuldade_maxima = df['NU_PARAM_B'].max()

    df['dificuldade_normalizada'] = 1 + ((df['NU_PARAM_B'] - dificuldade_minima) / (dificuldade_maxima - dificuldade_minima)) * 9

    df = df.drop_duplicates(subset=['CO_ITEM'], keep='last') # Remove Questões duplicadas (Diferentes cadernos)

    df['TP_LINGUA'] = df['TP_LINGUA'].fillna("PT").replace(0, "EN").replace(1, "ES")
    
    return df[['SG_AREA', 'CO_POSICAO', 'CO_ITEM','CO_HABILIDADE', 'TP_LINGUA','TX_GABARITO', 'dificuldade_normalizada']].rename(columns={
        'SG_AREA' : 'MATERIA',
        'CO_POSICAO': 'POSICAO',
        'CO_ITEM': 'COD_QUESTAO',
        'CO_HABILIDADE': 'HABILIDADE',
        'TP_LINGUA': 'LINGUA',
        'TX_GABARITO': 'RESPOSTA',
        'dificuldade_normalizada': 'DIFICULDADE'
    }).sort_values(['POSICAO']).reset_index(drop=True)

In [5]:
def extrair_texto_conteudo(content_list):
    """Extrai o texto de uma lista de conteúdo, concatenando textos e ignorando imagens"""
    textos = []
    for item in content_list:
        if item.get('type') == 'text':
            conteudo = item.get('content', '').strip()
            if conteudo:  # Só adiciona se não estiver vazio
                textos.append(conteudo)
    return ' '.join(textos)

def processar_questao_json(questao):
    """Processa uma questão do JSON e extrai enunciado e alternativas"""
    enunciado = extrair_texto_conteudo(questao.get('content', []))
    
    alternativas = {}
    for alt_key, alt_data in questao.get('alternatives', {}).items():
        letra = alt_data.get('alternative', '')
        texto_alt = extrair_texto_conteudo(alt_data.get('content', []))
        alternativas[letra] = texto_alt
    
    return {
        'enunciado': enunciado,
        'alternativas': alternativas
    }


In [6]:
def carregar_questoes_json(ano):
    """Carrega e processa as questões dos arquivos JSON"""
    questoes_d1 = []
    questoes_d2 = []
    
    # Carregar d1.json (LC + CH)
    with open(f'{ano}/d1.json', 'r', encoding='utf-8') as f:
        data_d1 = json.load(f)
        for questao in data_d1['data']:
            questao_processada = processar_questao_json(questao)
            questao_processada['number'] = questao['number']
            questoes_d1.append(questao_processada)
    
    # Carregar d2.json (CN + MT)
    with open(f'{ano}/d2.json', 'r', encoding='utf-8') as f:
        data_d2 = json.load(f)
        for questao in data_d2['data']:
            questao_processada = processar_questao_json(questao)
            questao_processada['number'] = questao['number']
            questoes_d2.append(questao_processada)
    
    return questoes_d1, questoes_d2


In [7]:
def mapear_questao_para_json(row, questoes_d1, questoes_d2):
    """Mapeia uma linha do CSV para a questão correspondente no JSON"""
    materia = row['MATERIA']
    posicao = row['POSICAO']
    lingua = row['LINGUA']
    
    # Determinar qual JSON usar baseado na matéria
    if materia in ['LC', 'CH']:
        questoes_json = questoes_d1
    else:  # CN, MT
        questoes_json = questoes_d2
    
    # Para LC, tratar questões de língua estrangeira
    if materia == 'LC' and posicao <= 5:
        # Para questões 1-5, precisamos encontrar a questão correta baseada na língua
        # No JSON, as questões de inglês e espanhol têm o mesmo número, mas aparecem em sequência
        questoes_numero = [q for q in questoes_json if q.get('number') == posicao]
        
        if lingua == 'EN':
            # Primeira questão com esse número (inglês)
            if questoes_numero:
                return questoes_numero[0]
        elif lingua == 'ES':
            # Segunda questão com esse número (espanhol)
            if len(questoes_numero) > 1:
                return questoes_numero[1]
        else:
            # Português - questões 6+
            for q in questoes_json:
                if q.get('number') == posicao:
                    return q
    else:
        # Para outras matérias ou questões 6+ de LC, posição = número da questão
        for q in questoes_json:
            if q.get('number') == posicao:
                return q
    
    return {}


In [8]:
def processa_prova_completa(ano):
    """Processa a prova completa, cruzando dados do CSV com dados dos JSONs"""
    # Carregar dados do CSV
    df = processa_prova(ano)
    
    # Carregar dados dos JSONs
    questoes_d1, questoes_d2 = carregar_questoes_json(ano)
    
    # Adicionar colunas de enunciado e alternativas
    df['ENUNCIADO'] = ''
    df['ALT_A'] = ''
    df['ALT_B'] = ''
    df['ALT_C'] = ''
    df['ALT_D'] = ''
    df['ALT_E'] = ''
    
    # Processar cada linha
    for idx, row in df.iterrows():
        questao_json = mapear_questao_para_json(row, questoes_d1, questoes_d2)
        
        if questao_json:
            df.at[idx, 'ENUNCIADO'] = questao_json.get('enunciado', '')
            alternativas = questao_json.get('alternativas', {})
            df.at[idx, 'ALT_A'] = alternativas.get('A', '')
            df.at[idx, 'ALT_B'] = alternativas.get('B', '')
            df.at[idx, 'ALT_C'] = alternativas.get('C', '')
            df.at[idx, 'ALT_D'] = alternativas.get('D', '')
            df.at[idx, 'ALT_E'] = alternativas.get('E', '')
    
    return df


In [ ]:
# Testar a nova função
print("Carregando dados...")
df_completo = processa_prova_completa(2023)
print("Dados carregados com sucesso!")
print("\nColunas disponíveis:", df_completo.columns.tolist())
print(f"\nTotal de questões processadas: {len(df_completo)}")

Carregando dados...
Dados carregados com sucesso!

Colunas disponíveis: ['MATERIA', 'POSICAO', 'COD_QUESTAO', 'HABILIDADE', 'LINGUA', 'RESPOSTA', 'DIFICULDADE', 'ENUNCIADO', 'ALT_A', 'ALT_B', 'ALT_C', 'ALT_D', 'ALT_E']

Total de questões processadas: 185

=== QUESTÕES DE LINGUAGENS (LC) ===

Questão 1 - ES
Enunciado: TEXTO I ? PorQUÉ ME CUESTA TANTO ESTUDIAR? pORQUÉ ME CUESTA TANTO CONCENTRARME? PoRQUÉ...... pORQUÉ....
Alternativa A: olhar diferenciado para com o outro gera mudanças....

Questão 1 - EN
Enunciado: The average american tosses 300 pounds of food each year, making food the number one contributor to ...
Alternativa A: os lixões precisam de ampliação....

Questão 2 - ES
Enunciado: Me niego rotundamente A negar mi voz, Mi sangre y mi piel. Y me niego rotundamente A dejar de ser yo...
Alternativa A: advérbios como “ rotundamente ” e “ categóricament...

=== QUESTÕES DE CIÊNCIAS HUMANAS (CH) ===

Questão 46
Enunciado: Fotografia da avó bordada SCARELI, G. A máquina de costura e

In [ ]:
# Salvar o resultado em CSV
df_completo.to_csv('prova_enem_2023_completa_com_textos.csv', index=False, encoding='utf-8')
df_completo.to_json('prova_enem_2023_completa_com_textos.jsonl', orient='records', lines=True)

Arquivo salvo como 'prova_enem_2023_completa_com_textos.csv'

=== ESTATÍSTICAS FINAIS ===
Total de questões: 185
Questões por matéria:
  45: 0 questões
  45: 0 questões
  50: 0 questões
  45: 0 questões

Questões de LC por língua:
  5: 0 questões
  5: 0 questões
  40: 0 questões


In [13]:
print(processa_prova(2023).head(200))

    MATERIA  POSICAO  COD_QUESTAO  HABILIDADE LINGUA RESPOSTA  DIFICULDADE
0        LC        1       140765           6     ES        A     4.465337
1        LC        1       140743           6     EN        B     3.962507
2        LC        2       141707           7     ES        A     7.797528
3        LC        2       140595           5     EN        B     6.265619
4        LC        3       140736           8     EN        D     5.476049
..      ...      ...          ...         ...    ...      ...          ...
180      MT      176        81742           2     PT        E     6.408696
181      MT      177        14389          15     PT        A     6.798113
182      MT      178       111754          15     PT        B     9.657790
183      MT      179        14366           1     PT        C     4.865163
184      MT      180       125969          30     PT        C     5.104088

[185 rows x 7 columns]
